# MACHINE LEARNING EN SPARK

## Rice MSC Dataset

### OBJETIVO
La finalidad del proyecto es lograr un modelo de predicción capaz de clasificar cada instancia referente a un grano de arroz en los cinco tipos de variedad de arroz observados:

* Arborio
* Basmati
* Ipsala
* Jasmine
* Karacadag

### INTRODUCCIÓN
Se realizaron procesos de extracción de características basados en las técnicas de tratamiento de imágenes utilizando características morfológicas, de forma y de color para cinco variedades diferentes de arroz de la misma marca. Se obtuvo un total de 75 mil piezas de granos de arroz, incluidas 15 mil piezas de cada variedad de arroz. Se aplicaron operaciones de preprocesamiento a las imágenes y se dispusieron para la extracción de características. Se dedujeron un total de 106 características de las imágenes; 12 características morfológicas y 4 características de forma obtenidas mediante características morfológicas y 90 características de color obtenidas a partir de cinco espacios de color diferentes (RGB, HSV, Lab*, YCbCr, XYZ). Además, para las 106 características obtenidas, se seleccionaron características mediante pruebas ANOVA, X2 y Gain Ratio y se determinaron las características útiles. En todas las pruebas, de los 106 rasgos, se obtuvieron los 5 rasgos más eficaces y específicos redondez, compacidad, factor de forma 3, relación de aspecto y excentricidad. Los rasgos de color se enumeraron en distinto orden siguiendo estos rasgos.

### ANÁLISIS EXPLORATORIO DE DATOS

#### Cargamos los datos

In [83]:
df = spark.read.load("gs://bucket_fms/datos_proyecto/rice-msc-dataset.csv",format = "csv", header = True)

#### Vemos cuantas filas tiene el DataFrame

In [84]:
df.count()

75000

#### Eliminamos las filas con valores faltantes

In [85]:
df = df.dropna()

#### Vemos ahora con cuantas filas nos hemos quedado

In [86]:
df.count()

74992

Hay pocas filas con valores nulos, así que podemos asumir la pérdida de los datos. Si tuvieramos muchas tendríamos que sustituir esos valores faltantes por la media/mediana de la variable u otro tipo de técnicas utilizadas sobre valores faltantes.

#### Vemos cuantas variables contienen los datos

In [87]:
len(df.columns) # variables

107

#### El siguiente paso será transformar el tipo de datos de cada variable a su tipo de datos correcto con la ayuda de los metadatos

In [88]:
df.dtypes # observamos como se encuentran inicialmente

[('AREA', 'string'),
 ('PERIMETER', 'string'),
 ('MAJOR_AXIS', 'string'),
 ('MINOR_AXIS', 'string'),
 ('ECCENTRICITY', 'string'),
 ('EQDIASQ', 'string'),
 ('SOLIDITY', 'string'),
 ('CONVEX_AREA', 'string'),
 ('EXTENT', 'string'),
 ('ASPECT_RATIO', 'string'),
 ('ROUNDNESS', 'string'),
 ('COMPACTNESS', 'string'),
 ('SHAPEFACTOR_1', 'string'),
 ('SHAPEFACTOR_2', 'string'),
 ('SHAPEFACTOR_3', 'string'),
 ('SHAPEFACTOR_4', 'string'),
 ('meanRR', 'string'),
 ('meanRG', 'string'),
 ('meanRB', 'string'),
 ('StdDevRR', 'string'),
 ('StdDevRG', 'string'),
 ('StdDevRB', 'string'),
 ('skewRR', 'string'),
 ('skewRG', 'string'),
 ('skewRB', 'string'),
 ('kurtosisRR', 'string'),
 ('kurtosisRG', 'string'),
 ('kurtosisRB', 'string'),
 ('entropyRR', 'string'),
 ('entropyRG', 'string'),
 ('entropyRB', 'string'),
 ('meanH', 'string'),
 ('meanS', 'string'),
 ('meanV', 'string'),
 ('StdDevH', 'string'),
 ('StdDevS', 'string'),
 ('StdDevV', 'string'),
 ('skewH', 'string'),
 ('skewS', 'string'),
 ('skewV', 's

#### Vemos que todas las variables tienen que ser reales menos las variables CONVEX_AREA (entero) y CLASS (categórica)

In [89]:
# Lista de columnas a convertir
cols_to_convert = df.columns
cols_to_convert.remove("CONVEX_AREA")
cols_to_convert.remove("CLASS")

In [91]:
# Crea una copia del DataFrame original
df_converted = df

# Itera sobre las columnas y aplica .astype() a cada una
for col in cols_to_convert:
    df_converted = df_converted.withColumn(col, df[col].cast("float"))


In [92]:
df_converted = df_converted.withColumn("CONVEX_AREA", df["CONVEX_AREA"].cast("integer"))

#### Pasamos cada clase de arroz a entero

In [93]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="CLASS", outputCol="CLASS_INT", handleInvalid="keep")
model = indexer.fit(df_converted)
df_indexed = model.transform(df_converted)

* 0: Arborio
* 1: Basmati
* 2: Karacadag
* 3: Jasmine
* 4: Ipsala

#### Extraemos las características de salida

In [94]:
#df = df_indexed
labels = df_indexed.select("CLASS").collect()
labels_int = df_indexed.select("CLASS_INT").collect()
df_features = df_indexed.drop("CLASS","CLASS_INT")
features = df_indexed.columns

In [95]:
selected_columns = ["ROUNDNESS", "COMPACTNESS", "SHAPEFACTOR_3", "ASPECT_RATIO", "ECCENTRICITY","CLASS_INT"]
filtered_df = df_indexed.select(selected_columns)

In [96]:
from pyspark.sql.functions import col
filtered_df = filtered_df.withColumn("CLASS_INT", col("CLASS_INT").cast("integer"))

In [97]:
filtered_df.show(10)

+---------+-----------+-------------+------------+------------+---------+
|ROUNDNESS|COMPACTNESS|SHAPEFACTOR_3|ASPECT_RATIO|ECCENTRICITY|CLASS_INT|
+---------+-----------+-------------+------------+------------+---------+
|   0.5114|     0.4751|       0.2257|      4.3693|      0.9735|        1|
|    0.812|     0.7065|       0.4992|      1.9807|      0.8632|        0|
|   0.6505|     0.5689|       0.3236|      3.0482|      0.9447|        3|
|   0.5256|     0.5007|       0.2507|      3.9325|      0.9671|        1|
|   0.7944|     0.6932|       0.4806|      2.0519|      0.8732|        0|
|   0.7374|     0.6824|       0.4656|      2.1013|      0.8795|        4|
|   0.4722|     0.4496|       0.2021|      4.8441|      0.9785|        1|
|   0.7992|      0.713|       0.5083|      1.9408|       0.857|        0|
|   0.6673|      0.595|       0.3541|      2.7618|      0.9321|        3|
|    0.904|     0.7993|       0.6389|      1.5539|      0.7654|        2|
+---------+-----------+-------------+-

#### Estandarizamos

In [98]:
featureCols = filtered_df.columns
del featureCols[-1]

In [99]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

In [100]:
# put features into a feature vector column
assembler = VectorAssembler(inputCols=featureCols, outputCol="features_raw") 
assembled_df = assembler.transform(filtered_df)

In [101]:
assembled_df.show(10)

+---------+-----------+-------------+------------+------------+---------+--------------------+
|ROUNDNESS|COMPACTNESS|SHAPEFACTOR_3|ASPECT_RATIO|ECCENTRICITY|CLASS_INT|        features_raw|
+---------+-----------+-------------+------------+------------+---------+--------------------+
|   0.5114|     0.4751|       0.2257|      4.3693|      0.9735|        1|[0.51139998435974...|
|    0.812|     0.7065|       0.4992|      1.9807|      0.8632|        0|[0.81199997663497...|
|   0.6505|     0.5689|       0.3236|      3.0482|      0.9447|        3|[0.65049999952316...|
|   0.5256|     0.5007|       0.2507|      3.9325|      0.9671|        1|[0.52560001611709...|
|   0.7944|     0.6932|       0.4806|      2.0519|      0.8732|        0|[0.79439997673034...|
|   0.7374|     0.6824|       0.4656|      2.1013|      0.8795|        4|[0.73739999532699...|
|   0.4722|     0.4496|       0.2021|      4.8441|      0.9785|        1|[0.47220000624656...|
|   0.7992|      0.713|       0.5083|      1.9408|

In [102]:
standardScaler = StandardScaler(inputCol="features_raw", outputCol="features")

In [103]:
scaled_df = standardScaler.fit(assembled_df).transform(assembled_df)

In [104]:
df_final = scaled_df.select("CLASS_INT","features")

In [107]:
df_final.show(10)

+---------+--------------------+
|CLASS_INT|            features|
+---------+--------------------+
|        1|[3.68863331914710...|
|        0|[5.85680535894486...|
|        3|[4.69193595175871...|
|        1|[3.79105551679089...|
|        0|[5.72985982110623...|
|        4|[5.31872951796710...|
|        1|[3.40589114120370...|
|        0|[5.76448148775941...|
|        3|[4.81311114016893...|
|        2|[6.52038436536277...|
+---------+--------------------+
only showing top 10 rows



#### Dividimos el conjunto en train y test

In [108]:
# Dividir el DataFrame "df" en dos conjuntos, uno de entrenamiento (80%) y otro de prueba (20%)
train_data, test_data = df_final.sample(False, 0.8), scaled_df.sample(False, 0.2)

In [109]:
train_data.columns

['CLASS_INT', 'features']

### ENTRENAMOS EL MODELO

**Métrica utilizada:** En problemas de clasificación con múltiples clases, una métrica comúnmente utilizada es la precisión, que mide cuántas predicciones hechas por el modelo son correctas.

#### **REGRESIÓN LOGÍSTICA**

In [110]:
# Importar la clase LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Crear una instancia del modelo
lr = LogisticRegression(labelCol='CLASS_INT')

# Ajustar el modelo al conjunto de entrenamiento
model_lr = lr.fit(train_data)

# Hacer predicciones en el conjunto de test
predictions_lr = model_lr.transform(test_data)

# Evaluar el rendimiento del modelo
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
print('Accuracy: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='accuracy').evaluate(predictions_lr))
print('Precision: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_lr))


Accuracy:  0.8929240235420011


Precision:  0.8924346485386854


**Logramos una precisión de 0.892 con un modelo sencillo, lo que es una buena señal.**

#### **DECISION TREE**

In [126]:
# Importar la clase DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassifier

# Crear una instancia del modelo
precis_dt = 0
for i in range(2,11):
    dt = DecisionTreeClassifier(labelCol='CLASS_INT',maxDepth=i)

    # Ajustar el modelo al conjunto de entrenamiento
    model_dt = dt.fit(train_data)

    # Hacer predicciones en el conjunto de test
    predictions_dt = model_dt.transform(test_data)

    # Evaluar el rendimiento del modelo
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    print("maxDepth:",i, 'Accuracy: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='accuracy').evaluate(predictions_dt),
          'Precision: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_dt))
    
    if MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_dt) > precis_dt:
        precis = MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_dt)
        modelo_dt = model_dt


maxDepth: 2 Accuracy:  0.7774879614767255 Precision:  0.6692129417473281


maxDepth: 3 Accuracy:  0.8832263242375602 Precision:  0.8816088158381881


maxDepth: 4 Accuracy:  0.8878410914927769 Precision:  0.8892651864110221


maxDepth: 5 Accuracy:  0.889446227929374 Precision:  0.8915924074242241


maxDepth: 6 Accuracy:  0.8909176029962547 Precision:  0.8916803872706333


maxDepth: 7 Accuracy:  0.8919208132691279 Precision:  0.8926393720497585


maxDepth: 8 Accuracy:  0.8924558587479936 Precision:  0.8930025417873692


maxDepth: 9 Accuracy:  0.8923889780631353 Precision:  0.8930853433202905


maxDepth: 10 Accuracy:  0.89258962011771 Precision:  0.8931902940076617


In [132]:
print("Modelo:",modelo_dt)
print("Precisión:",precis)

Modelo: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a277d8f3be6d, depth=10, numNodes=189, numClasses=5, numFeatures=5
Precisión: 0.8931902940076617


**El modelo con una profundidad de 10 es el que mejor resultado proporciona con una precisión de 0.893.**

#### **RANDOM FOREST**

In [133]:
# Importar la clase RandomForestClassifier
from pyspark.ml.classification import RandomForestClassifier

precis_rf = 0

for i in range(15,30):
    
    rf = RandomForestClassifier(labelCol='CLASS_INT',numTrees=i)

    # Ajustar el modelo al conjunto de entrenamiento
    model_rf = rf.fit(train_data)

    # Hacer predicciones en el conjunto de test
    predictions_rf = model_rf.transform(test_data)

    # Evaluar el rendimiento del modelo
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    print("numTrees:",i, 'Accuracy: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='accuracy').evaluate(predictions_rf),
          'Precision: ', MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_rf))
    print()
    if MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='accuracy').evaluate(predictions_rf) > precis_rf:
        precis_rf = MulticlassClassificationEvaluator(labelCol='CLASS_INT', metricName='weightedPrecision').evaluate(predictions_dt)
        modelo_rf = model_rf


numTrees: 15 Accuracy:  0.8857677902621723 Precision:  0.8868581889874035



numTrees: 16 Accuracy:  0.8893793472445158 Precision:  0.8895714002489461



numTrees: 17 Accuracy:  0.8859015516318887 Precision:  0.8868662070594634



numTrees: 18 Accuracy:  0.8857677902621723 Precision:  0.8867107074020725



numTrees: 19 Accuracy:  0.8860353130016051 Precision:  0.8868852267432172



...

numTrees: 25 Accuracy:  0.8857677902621723 Precision:  0.8866525968817732



numTrees: 26 Accuracy:  0.8859684323167469 Precision:  0.8869128020971637



numTrees: 27 Accuracy:  0.8860353130016051 Precision:  0.8868830120198942



numTrees: 28 Accuracy:  0.8859015516318887 Precision:  0.886774788775389



numTrees: 29 Accuracy:  0.8856340288924559 Precision:  0.8865960063681685



In [134]:
print("Modelo:",modelo_rf)
print("Precisión:",precis_rf)

Modelo: RandomForestClassificationModel: uid=RandomForestClassifier_ea4d3e26f22b, numTrees=15, numClasses=5, numFeatures=5
Precisión: 0.8931902940076617


**El modelo de 15 árboles consigue una precisión de 0.893**

## CONCLUSIÓN

**Comparamos la precisión de los modelos**

* Regresión Logística: 0.8924
* Decision Tree: 0.8931
* Random Forest: 0.8931

Los tres modelos obtienen unas métricas casi iguales.

En la práctica escogeríamos la **regresión lógistica** por ser el modelo que más simple que nos proporciona buenos resultados.

Además podemos tener en cuenta la **exactitud** del modelo, y realizando la comparación (0.8929 vs. 0.8925 vs. 0.885), se observa como la regresión logística sigue siendo la mejor opción.